# MODEL 5 

This notebook shows the implemnetation of the model 5. See the documenation for further details.
#### The preprocessing we apply is the simple one; 
#### The Doc2vec model is trained on all violations samples, all the train and tes20 samples and 90% of the train and test20 of the N-th article;
#### We applay on the SVM model 10-fold cross-validation, which means it is trained on  9-folds of the dataset (train+test20) and tested on the remaining 1-fold  (seperately for each article).

In [1]:
import os
import sys
import shutil
import random
import re
import regex
import string
from random import shuffle
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from gensim import models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import svm, metrics
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from sklearn.preprocessing import normalize
from module_preprocessing import apply_preprocessing,merge_dataset,create_vector_dataset
from ast import literal_eval
from time import time
from datetime import timedelta
from sklearn.model_selection import ParameterGrid

random_seed = [6789, 232345, 334456, 454567, 567448, 67839, 9822276, 763454, 644543, 3454421]

In [2]:
path = "crystal_ball_data/SIMPLE_PREP/violations_RAW_DATASET/"
violations = []
for filename in os.listdir(path):
    dataset = pd.read_csv(path + filename, index_col="index")
    dataset.raw_text = dataset.raw_text.apply(literal_eval)
    random.seed(6789)
    dataset = dataset.sample(frac=1, random_state=6789).reset_index(drop=True)
    violations.append([dataset, filename[:9]])
    print("dataset " + filename[:9] + " has been loaded. Shape: ",dataset.shape)

dataset Article02 has been loaded. Shape:  (397, 2)
dataset Article06 has been loaded. Shape:  (4090, 2)
dataset Article13 has been loaded. Shape:  (1059, 2)
dataset Article10 has been loaded. Shape:  (251, 2)
dataset Article08 has been loaded. Shape:  (495, 2)
dataset Article05 has been loaded. Shape:  (1117, 2)
dataset Article11 has been loaded. Shape:  (88, 2)
dataset Article03 has been loaded. Shape:  (850, 2)
dataset Article14 has been loaded. Shape:  (43, 2)


In [3]:
path = "crystal_ball_data/TT20_simple_rd/simple/"
train_test20 = []
for filename in os.listdir(path):
    dataset = pd.read_csv(path + filename, index_col="index")
    dataset.raw_text = dataset.raw_text.apply(literal_eval)
    random.seed(6789)
    dataset = dataset.sample(frac=1, random_state=6789).reset_index(drop=True)
    train_test20.append([dataset, filename[:9]])
    print("dataset " + filename[:9] + " has been loaded. Shape: ",dataset.shape)

dataset Article02 has been loaded. Shape:  (140, 2)
dataset Article06 has been loaded. Shape:  (1142, 2)
dataset Article13 has been loaded. Shape:  (263, 2)
dataset Article10 has been loaded. Shape:  (264, 2)
dataset Article08 has been loaded. Shape:  (570, 2)
dataset Article05 has been loaded. Shape:  (374, 2)
dataset Article11 has been loaded. Shape:  (78, 2)
dataset Article03 has been loaded. Shape:  (709, 2)
dataset Article14 has been loaded. Shape:  (358, 2)


In [8]:
def fit(parameter,df, filename, n_shuffles=10, n_splits=10, verbose=0):

    matr_accuracy = np.zeros((n_shuffles, n_splits), dtype=float)
    if verbose >= 2:
        print("SEEDs: ", random_seed)

    for i in range(n_shuffles):

        if verbose >= 1:
            print("\n_______Shuffle number " + str(i) + "_______\nSEED:" +
                  str(random_seed[i]))
        random.seed(random_seed[i])
        df = df.sample(
            frac=1, random_state=random_seed[i]).reset_index(drop=True)
        if verbose >= 2:
            print("SHUFFLED DATASET: \n")
            print(df.head(3))
            print(df.shape)
        kf = KFold(n_splits=n_splits)
        count = 0
        for train_index, test_index in kf.split(df):
            if verbose >= 1:
                print("_______cross validation round: " + str(count) +
                      "_______")
            if verbose >= 3: print("test indexes: ", test_index)

            df_test = df.loc[test_index[0]:test_index[len(test_index) - 1]]
            df_train = df.drop(df.index[test_index])
            if verbose >= 3:
                print("TEST SET: ", df_test)
                print("shape: ", df_test.shape)
                print("TRAIN SET:", df_train.head(10))
                print("shape: ", df_train.shape)

            data_all = pd.DataFrame()
            data_all =  data_all.append(df_train)
            for x in violations:
                data_all = data_all.append(x[0])
            for x in train_test20 :
                if filename[:9] != x[1]:
                    data_all = data_all.append(x[0])
            start = time()
            documents = []
            for index, sample in data_all.iterrows():
                documents.append(
                    TaggedDocument(
                        words=sample['raw_text'], tags=[sample['tag']]))
            if verbose >= 0: 
                print("Training Doc2Vec with ",len(documents)," samples.")
                print("Training SVM with ",len(df_train)," samples.")
                print("Testing SVM on ",len(df_test)," samples.")
            model_D2V = Doc2Vec(
                documents,
                negative=parameter['negative'],
                ns_exponent=parameter['ns_exponent'],
                hs=parameter['hs'],
                window=parameter['window'],
                dm_mean=parameter['dm_mean'],
                dm_concat=0,
                dbow_words=parameter['dbow_words'],
                vector_size=parameter['vector_size'],
                epochs=parameter['epochs'],
                min_count=parameter['min_count'],
                workers=os.cpu_count())

            df_vector = create_vector_dataset(df_train, model_D2V)
            df_test_vector = create_vector_dataset(df_test, model_D2V)
            if verbose >= 2:
                print("TEST SET VECTOR: ", df_test_vector.head(10))
                print("shape: ", df_test_vector.shape)
                print("TRAIN SET:", df_vector.head(10))
                print("shape: ", df_vector.shape)
            if verbose >= 0: 
                print("model Doc2Vec created. Time elasped: " +
                      str(timedelta(seconds=(time() - start))))
            clf_svm = LinearSVC(C=1)
            clf_svm.fit(
                df_vector.loc[:, 'y_0':'y_' + str(parameter['vector_size'] - 1)].values,
                df_vector.loc[:, 'tag'].values)

            predictions = clf_svm.predict(
                df_test_vector.loc[:, 'y_0':'y_' +
                                   str(parameter['vector_size'] - 1)].values)
            tag_test = df_test_vector.loc[:, 'tag'].values
            acc = metrics.accuracy_score(tag_test, predictions)
            if verbose >= 0:
                print("ACCURACY:", acc)
                #print("Precision:",metrics.precision_score(tag_test, predictions))
                #print("Recall:",metrics.recall_score(tag_test, predictions))

            matr_accuracy[i][count] = acc
            count += 1

    return matr_accuracy

In [9]:
param_grid = {
            'vector_size': [100],
            'epochs': [20],
            'min_count': [1],
            'window': [3],
            'hs': [1],
            'negative': [20],
            'ns_exponent': [1],
            'dm_mean': [1],
            'dbow_words': [1]
}


In [11]:
if __name__ == "__main__":
    print(
        "Note: \n* Be sure you are in the same folder contaning the Crystal Ball data set.\n* You might need to install some libraries, here's the list: "
    )
    summary=[]
    print(set(sys.modules) & set(globals()))
    path = "crystal_ball_data/TT20_simple_rd/simple/"
    parameters = list(ParameterGrid(param_grid))
    print("\nTotal combination of parameters: %d" % len(parameters))
    for i,parameter in enumerate(parameters) :
        results=[]
        print("Iteration parameter ", i)
        for filename in os.listdir(path):
            print("\n\n____",filename[:9],"____")
            dataframe = pd.read_csv(path + filename, index_col="index")
            dataframe.raw_text = dataframe.raw_text.apply(literal_eval)
            %time matr = fit(parameter,dataframe,filename,n_shuffles=10,n_splits=10,verbose=1)
            results.append([matr,filename,len(dataframe)])
            print("mean: ", matr.mean())
            print("min: ", matr.min())
            print("max: ", matr.max())
        total=0
        for res in results : total+=res[0].mean()
        print("parameters:", parameter)
        print("TOTAL ACCURACY: ",total/len(results))
        summary.append([results,parameter])
    print(results)

Note: 
* Be sure you are in the same folder contaning the Crystal Ball data set.
* You might need to install some libraries, here's the list: 
{'regex', 'string', 'shutil', 're', 'sys', 'time', 'nltk', 'os', 'random'}

Total combination of parameters: 1
Iteration parameter  0


____ Article02 ____

_______Shuffle number 0_______
SEED:6789
_______cross validation round: 0_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:16:51.388962
ACCURACY: 0.6428571428571429
_______cross validation round: 1_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:16:35.854175
ACCURACY: 0.6428571428571429
_______cross validation round: 2_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:16:41.716034
ACCURACY: 0.5714285714

Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:17:53.271652
ACCURACY: 0.7142857142857143
_______cross validation round: 6_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:17:01.383841
ACCURACY: 0.6428571428571429
_______cross validation round: 7_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:16:54.686507
ACCURACY: 0.7142857142857143
_______cross validation round: 8_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:16:45.578646
ACCURACY: 0.7142857142857143
_______cross validation round: 9_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2V

model Doc2Vec created. Time elasped: 0:19:13.147036
ACCURACY: 0.5714285714285714
_______cross validation round: 2_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:18:36.402476
ACCURACY: 0.9285714285714286
_______cross validation round: 3_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:18:57.277076
ACCURACY: 0.9285714285714286
_______cross validation round: 4_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:19:10.435595
ACCURACY: 0.9285714285714286
_______cross validation round: 5_______
Training Doc2Vec with  12274  samples.
Training SVM with  126  samples.
Testing SVM on  14  samples.
model Doc2Vec created. Time elasped: 0:19:14.105130
ACCURACY: 0.5714285714285714
_______cross validation round: 

Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:24:13.687967
ACCURACY: 0.6666666666666666
_______cross validation round: 8_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:21:38.484342
ACCURACY: 0.7192982456140351
_______cross validation round: 9_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:21:24.814531
ACCURACY: 0.6491228070175439

_______Shuffle number 1_______
SEED:232345
_______cross validation round: 0_______
Training Doc2Vec with  12173  samples.
Training SVM with  1027  samples.
Testing SVM on  115  samples.
model Doc2Vec created. Time elasped: 0:22:04.616190
ACCURACY: 0.6956521739130435
_______cross validation round: 1_______
Training Doc2Vec with  12173  samples.
Training SVM with  10

Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:24:00.350642
ACCURACY: 0.5877192982456141
_______cross validation round: 4_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:26:07.000237
ACCURACY: 0.7192982456140351
_______cross validation round: 5_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:25:14.367873
ACCURACY: 0.6754385964912281
_______cross validation round: 6_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
model Doc2Vec created. Time elasped: 0:24:04.667569
ACCURACY: 0.6842105263157895
_______cross validation round: 7_______
Training Doc2Vec with  12174  samples.
Training SVM with  1028  samples.
Testing SVM on  114  samples.
m

...

model Doc2Vec created. Time elasped: 0:31:05.921984
ACCURACY: 0.7605633802816901
_______cross validation round: 7_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:31:16.411308
ACCURACY: 0.8169014084507042
_______cross validation round: 8_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:31:39.531339
ACCURACY: 0.7464788732394366
_______cross validation round: 9_______
Training Doc2Vec with  12218  samples.
Training SVM with  639  samples.
Testing SVM on  70  samples.
model Doc2Vec created. Time elasped: 0:31:30.202555
ACCURACY: 0.8

_______Shuffle number 3_______
SEED:454567
_______cross validation round: 0_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:30:58.415659
ACCURACY: 0.7887323943661971
__

Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:29:58.571076
ACCURACY: 0.7464788732394366
_______cross validation round: 4_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:30:24.243453
ACCURACY: 0.7464788732394366
_______cross validation round: 5_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:30:06.502821
ACCURACY: 0.704225352112676
_______cross validation round: 6_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Vec created. Time elasped: 0:29:41.041911
ACCURACY: 0.7183098591549296
_______cross validation round: 7_______
Training Doc2Vec with  12217  samples.
Training SVM with  638  samples.
Testing SVM on  71  samples.
model Doc2Ve

model Doc2Vec created. Time elasped: 0:31:32.715213
ACCURACY: 0.7571428571428571
CPU times: user 8d 1h 13min 58s, sys: 1h 19min 9s, total: 8d 2h 33min 8s
Wall time: 2d 2h 16min 19s
mean:  0.7588148893360162
min:  0.6338028169014085
max:  0.8873239436619719


____ Article14 ____

_______Shuffle number 0_______
SEED:6789
_______cross validation round: 0_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:27:48.996265
ACCURACY: 0.6666666666666666
_______cross validation round: 1_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:26:53.580540
ACCURACY: 0.8611111111111112
_______cross validation round: 2_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:26:18.711850
ACCURACY: 0.6666666666666666
_______cross 

Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:25:24.281850
ACCURACY: 0.6944444444444444
_______cross validation round: 6_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:26:31.811428
ACCURACY: 0.6388888888888888
_______cross validation round: 7_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:26:39.991799
ACCURACY: 0.75
_______cross validation round: 8_______
Training Doc2Vec with  12253  samples.
Training SVM with  323  samples.
Testing SVM on  35  samples.
model Doc2Vec created. Time elasped: 0:26:06.437117
ACCURACY: 0.8571428571428571
_______cross validation round: 9_______
Training Doc2Vec with  12253  samples.
Training SVM with  323  samples.
Testing SVM on  35  samples.
model Doc2Vec created. Ti

Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:28:23.548248
ACCURACY: 0.8055555555555556
_______cross validation round: 3_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:28:30.584062
ACCURACY: 0.6944444444444444
_______cross validation round: 4_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:28:00.666526
ACCURACY: 0.6944444444444444
_______cross validation round: 5_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2Vec created. Time elasped: 0:28:14.414536
ACCURACY: 0.8333333333333334
_______cross validation round: 6_______
Training Doc2Vec with  12252  samples.
Training SVM with  322  samples.
Testing SVM on  36  samples.
model Doc2V

In [12]:
res= summary[0][0] 

In [16]:
path="results/"
for r in res : 
    np.savetxt(path+r[1][:9], r[0])

In [18]:
np.loadtxt("results/Article02")

array([[0.64285714, 0.64285714, 0.57142857, 0.71428571, 0.57142857,
        0.92857143, 0.57142857, 0.64285714, 0.71428571, 0.92857143],
       [0.71428571, 0.85714286, 0.57142857, 0.57142857, 0.57142857,
        0.57142857, 0.71428571, 0.64285714, 0.64285714, 0.78571429],
       [0.71428571, 0.78571429, 0.85714286, 0.57142857, 0.57142857,
        0.92857143, 0.85714286, 0.57142857, 0.64285714, 0.57142857],
       [0.64285714, 0.64285714, 0.85714286, 0.64285714, 0.78571429,
        0.71428571, 0.64285714, 0.71428571, 0.71428571, 0.92857143],
       [0.57142857, 0.71428571, 0.5       , 0.78571429, 0.57142857,
        0.85714286, 0.78571429, 0.64285714, 0.5       , 0.78571429],
       [0.71428571, 0.85714286, 0.57142857, 0.92857143, 0.71428571,
        0.85714286, 0.5       , 0.71428571, 0.57142857, 0.85714286],
       [0.78571429, 1.        , 0.71428571, 0.78571429, 0.78571429,
        0.78571429, 0.78571429, 0.57142857, 0.35714286, 0.42857143],
       [0.85714286, 0.57142857, 0.9285714

In [54]:
#cv=3 conf 2
tot=0
for res in results[63:72] : 
    print(res[1][:9]," Accuracy: %0.3f"% res[0].mean(),"samples SVM training: ",(res[2]/3)*2)
    tot+= res[0].mean()
print(tot/9)
print(len(results))

Article02  Accuracy: 0.672 samples SVM training:  93.33333333333333
Article06  Accuracy: 0.676 samples SVM training:  761.3333333333334
Article13  Accuracy: 0.779 samples SVM training:  175.33333333333334
Article10  Accuracy: 0.580 samples SVM training:  176.0
Article08  Accuracy: 0.672 samples SVM training:  380.0
Article05  Accuracy: 0.663 samples SVM training:  249.33333333333334
Article11  Accuracy: 0.705 samples SVM training:  52.0
Article03  Accuracy: 0.748 samples SVM training:  472.6666666666667
Article14  Accuracy: 0.737 samples SVM training:  238.66666666666666
0.6924950610094991
72


In [35]:
#cv=3 conf 2

Article02  Accuracy: 0.736 samples SVM training:  93.33333333333333
Article06  Accuracy: 0.637 samples SVM training:  761.3333333333334
Article13  Accuracy: 0.780 samples SVM training:  175.33333333333334
Article10  Accuracy: 0.652 samples SVM training:  176.0
Article08  Accuracy: 0.653 samples SVM training:  380.0
Article05  Accuracy: 0.685 samples SVM training:  249.33333333333334
Article11  Accuracy: 0.679 samples SVM training:  52.0
Article03  Accuracy: 0.760 samples SVM training:  472.6666666666667
Article14  Accuracy: 0.765 samples SVM training:  238.66666666666666
0.7050844015056899


In [29]:
#cv=3 conf 3 

Article02  Accuracy: 0.686 samples SVM training:  140
Article06  Accuracy: 0.687 samples SVM training:  1142
Article13  Accuracy: 0.783 samples SVM training:  263
Article10  Accuracy: 0.557 samples SVM training:  264
Article08  Accuracy: 0.677 samples SVM training:  570
Article05  Accuracy: 0.674 samples SVM training:  374
Article11  Accuracy: 0.782 samples SVM training:  78
Article03  Accuracy: 0.750 samples SVM training:  709
Article14  Accuracy: 0.704 samples SVM training:  358
0.7001068952554576
